<a href="https://colab.research.google.com/github/nh0875/Desafios-Procesamiento-del-Habla-974/blob/main/Copy_of_Desaf%C3%ADo_Final_Procesamiento_del_Habla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio Obligatorio Procesamiento del Habla**
**Juan Ignacio Hilliard Bochaca 4162974**

**Tema 5 (Análisis de Sentimiento del Texto)**

Introduccion:

In [ ]:
!pip install transformers PyPDF2 pdfplumber torch

In [ ]:
# Librerías necesarias
import pdfplumber
# re para limpiar el texto
import re
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, roc_auc_score, balanced_accuracy_score
from transformers import BertTokenizer, BertModel, AdamW
import pandas as pd
import random

In [ ]:

# Filtrado y obtención de estrofas
def filtrar_texto(texto):
    texto_filtrado=re.sub(r'\b[I|V|X|L|C|D|M]+\b', '', texto)
    texto_filtrado=re.sub(r'\b(Página|Capítulo)\s?\d+\b', '', texto_filtrado)
    return texto_filtrado

def obtener_estrofas(texto):
    lineas=[linea.strip() for linea in texto.split("\n") if linea.strip()]
    estrofas=[]
    i=0
    total_lineas=len(lineas)
    while i < total_lineas:
        if total_lineas - i == 8:
            estrofa="\n".join(lineas[i:i + 8])
            estrofas.append(estrofa)
            i += 8
        else:
            estrofa="\n".join(lineas[i:i + 6])
            estrofas.append(estrofa)
            i += 6
    return estrofas

In [ ]:
# Extraer texto del PDF
with pdfplumber.open("/content/hernandez_jose_-_el_gaucho_martin_fierro.pdf") as pdf:
    text=""
    for page_num in range(3, len(pdf.pages)):
        page=pdf.pages[page_num]
        page_text=page.extract_text()
        if page_text:
            text += page_text + "\n"

# Filtrado y división en estrofas
text=filtrar_texto(text)
estrofas=obtener_estrofas(text)

# Crear DataFrame y etiquetas simuladas
labels=[random.choice(['Positivo', 'Neutral', 'Negativo']) for _ in estrofas]
df=pd.DataFrame({'estrofa': estrofas, 'sentimiento': labels})

# Inicialización de BERT
tokenizer=BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
bert_model=BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

# Crear dataset personalizado
class EstrofaDataset(Dataset):
    def __init__(self, estrofas, labels):
        self.estrofas=estrofas
        self.labels=labels

    def __len__(self):
        return len(self.estrofas)

    def __getitem__(self, idx):
        encoding=tokenizer(self.estrofas[idx], padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        label=self.labels[idx]
        return {key: val.squeeze(0) for key, val in encoding.items()}, torch.tensor(label)

# Mapear las etiquetas
label_map={'Positivo': 1, 'Neutral': 0, 'Negativo': -1}
df['sentimiento']=df['sentimiento'].map(label_map)

# Dividir los datos
X_train, X_test, y_train, y_test=train_test_split(df['estrofa'].values, df['sentimiento'].values, test_size=0.2, random_state=42)
train_dataset=EstrofaDataset(X_train, y_train)
test_dataset=EstrofaDataset(X_test, y_test)

# Crear DataLoader
train_loader=DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=8)

# Definir modelo de clasificación basado en BERT
class BERTSentimentClassifier(nn.Module):
    def __init__(self, bert_model):
        super(BERTSentimentClassifier, self).__init__()
        self.bert=bert_model
        self.classifier=nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask):
        outputs=self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output=outputs.pooler_output
        return self.classifier(pooled_output)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Inicializar modelo y optimizador, despues se entrena al modelo
model=BERTSentimentClassifier(bert_model)
optimizer=AdamW(model.parameters(), lr=2e-5)
model.train()
for epoch in range(3):  # Número de épocas
    for batch in train_loader:
        optimizer.zero_grad()
        inputs, labels=batch
        outputs=model(inputs['input_ids'], inputs['attention_mask'])
        loss=nn.CrossEntropyLoss()(outputs, labels + 1)  # Ajuste de etiquetas
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, roc_auc_score
import pandas as pd

# Convertir listas a arreglos NumPy para operaciones vectorizadas
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Ajustar índices (si necesitas restar 1, esto depende de la naturaleza de tus etiquetas)
all_preds_adjusted = all_preds - 1

# Calcular métricas
print("Balanced Accuracy:", balanced_accuracy_score(all_labels, all_preds_adjusted))
print("Matthews Correlation Coefficient (MCC):", matthews_corrcoef(all_labels, all_preds_adjusted))
print("AUC-ROC:", roc_auc_score(pd.get_dummies(all_labels), pd.get_dummies(all_preds_adjusted), multi_class='ovo'))

Balanced Accuracy: 0.39697802197802196
Matthews Correlation Coefficient (MCC): 0.09689599192618088
AUC-ROC: 0.5479714421381088


In [ ]:
# Seleccionar 15 estrofas aleatorias del conjunto de prueba
indices_aleatorios=random.sample(range(len(X_test)), 15)

# Poner el modelo en modo de evaluación
model.eval()

print("\nClasificación de 15 estrofas seleccionadas aleatoriamente:\n")
with torch.no_grad():
    for i, idx in enumerate(indices_aleatorios, start=1):
        inputs=tokenizer(X_test[idx], padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        # Realizar la predicción
        outputs=model(inputs['input_ids'], inputs['attention_mask'])
        pred=torch.argmax(outputs, axis=1).item() - 1
        # Obtener la etiqueta en formato de texto
        sentimiento_predicho={1: "Positivo", 0: "Neutral", -1: "Negativo"}[pred]
        print(f"Estrofa {i}:")
        print(X_test[idx])
        print(f"Probabilidad de sentimiento predicho: {sentimiento_predicho}\n")


Clasificación de 15 estrofas seleccionadas aleatoriamente:

Estrofa 1:
la mujer, para olvidar.
Alcé mi poncho y mis prendas
y me largué a padecer
por culpa de una mujer
que quiso engañar a dos.
Al rancho le dije adiós,
Sentimiento Predicho: Neutral

Estrofa 2:
como agua de manantial.
Con la guitarra en la mano
ni las moscas se me arriman;
naides me pone el pie encima,
y, cuando el pecho se entona,
hago gemir a la prima
Sentimiento Predicho: Negativo

Estrofa 3:
Al verse sin compañero
el otro se sofrenó;
entonces le dentré yo,
sin dejarlo resollar,
pero ya empezó a aflojar
y a la pun...ta disparó.
Sentimiento Predicho: Positivo

Estrofa 4:
Para mi el campo son flores
dende que libre me veo;
donde me lleva el deseo
allí mis pasos dirijo
y hasta en las sombras, de fijo
que a dondequiera rumbeo.
Sentimiento Predicho: Neutral

Estrofa 5:
Sólo una manta peluda
era cuanto me quedaba;
la había agenciao a la taba
y ella me tapaba el bulto;
yaguané que allí ganaba
no salía... ni con indulto.
Se